In [29]:
#웹캠에서 영상을 읽어온다
cap = cv2.VideoCapture(0)
cap.set(3, 640) #WIDTH
cap.set(4, 480) #HEIGHT

#얼굴 인식 캐스케이드 파일 읽는다
face_cascade = cv2.CascadeClassifier('haarcascade_frontface.xml')

while(True):
    # frame 별로 capture 한다
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    #인식된 얼굴 갯수를 출력
    print(len(faces))

    # 인식된 얼굴에 사각형을 출력한다
    for (x,y,w,h) in faces:
         cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)

    #화면에 출력한다
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1
1
1
1
1


# 일단 되는 거 바로 아래것 근데 이상함


In [7]:
import cv2 as cv2
import math
import time



def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv2.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes


faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"

ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"

genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"


ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

# Load network
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)
faceNet = cv2.dnn.readNet(faceModel, faceProto)


#웹캠에서 영상을 읽어온다
cap = cv2.VideoCapture(0)
cap.set(3, 640) #WIDTH
cap.set(4, 480) #HEIGHT

#얼굴 인식 캐스케이드 파일 읽는다
face_cascade = cv2.CascadeClassifier('haarcascade_frontface.xml')

while(True):
    # frame 별로 capture 한다
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    #인식된 얼굴 갯수를 출력
    print(len(faces))

    # 인식된 얼굴에 사각형을 출력한다
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)

            
    frameFace, bboxes = getFaceBox(faceNet, frame)   # 얼굴찾는 함수
    padding = 10
    for bbox in bboxes:
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227),(78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv2.putText(frameFace, label, (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)        
        
    #화면에 출력한다
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.imwrite("age-gender-out.jpg", frameFace)
        break

cap.release()
cv2.destroyAllWindows()

1
Gender : Male, conf = 0.995
Age Output : [[1.6725605e-06 3.6393561e-05 9.6293896e-01 4.0181517e-03 3.2947931e-02
  4.9496415e-05 5.8362402e-06 1.4572593e-06]]
Age : (8-12), conf = 0.963
1
Gender : Male, conf = 0.997
Age Output : [[8.4271005e-07 6.5539636e-05 9.8994195e-01 2.2956629e-03 7.6797535e-03
  1.3612901e-05 2.1381234e-06 4.9333113e-07]]
Age : (8-12), conf = 0.990
1
Gender : Male, conf = 0.997
Age Output : [[1.0533052e-05 3.4926846e-04 7.2559434e-01 2.0366764e-02 2.5345132e-01
  1.9788998e-04 2.3769715e-05 6.0177258e-06]]
Age : (8-12), conf = 0.726
1
Gender : Male, conf = 0.981
Age Output : [[2.1688527e-05 1.8081801e-04 1.8741930e-01 1.6652426e-02 7.9467386e-01
  1.0015657e-03 3.4401543e-05 1.5935213e-05]]
Age : (25-32), conf = 0.795
1
Gender : Male, conf = 0.985
Age Output : [[2.4877654e-05 6.0366339e-04 3.7068254e-01 1.4871839e-02 6.1344147e-01
  3.4061409e-04 2.4371269e-05 1.0711863e-05]]
Age : (25-32), conf = 0.613
1
Gender : Male, conf = 0.971
Age Output : [[2.6180358e-06

신호가 들어올때 검출하는 저장키 같은것 만들자
아니면 시간이 어느정도 지낫을때의 조건을 줘서 저장하게


frame에 또 frame이 들어가서 2개가 생김

# 저장키값까지 성공

In [9]:
import cv2 as cv2
import math
import time



def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv2.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes


faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"

ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"

genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"


ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

# Load network
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)
faceNet = cv2.dnn.readNet(faceModel, faceProto)


#웹캠에서 영상을 읽어온다
cap = cv2.VideoCapture(0)
cap.set(3, 640) #WIDTH
cap.set(4, 480) #HEIGHT

#얼굴 인식 캐스케이드 파일 읽는다
face_cascade = cv2.CascadeClassifier('haarcascade_frontface.xml')

while(True):
    # frame 별로 capture 한다
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    #인식된 얼굴 갯수를 출력
    print(len(faces))

    # 인식된 얼굴에 사각형을 출력한다
    for (x1,y1,x2,y2) in faces:
        cv2.rectangle(frame,(x1,y1),(x1+x2,y1+y2),(255,0,0),2)

            
    frameFace, bboxes = getFaceBox(faceNet, frame)   # 얼굴찾는 함수
    padding = 10
    for bbox in bboxes:
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227),(78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv2.putText(frameFace, label, (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)        
        

    cv2.imshow('frame',frame)
    k = cv2.waitKey(5)
    if k == ord('q'): # esc key
        break
    elif k == ord('s'): # 's' key
        cv2.imwrite('testgray.png',frameFace)
        
cap.release()
cv2.destroyAllWindows()



1
Gender : Male, conf = 0.995
Age Output : [[3.0439827e-07 7.4303163e-05 9.9517041e-01 1.7305156e-03 3.0174558e-03
  5.3051526e-06 1.3066031e-06 3.7579707e-07]]
Age : (8-12), conf = 0.995
1
Gender : Male, conf = 0.998
Age Output : [[3.8576118e-07 4.8204292e-05 9.6821243e-01 5.1926835e-03 2.6520530e-02
  2.0001011e-05 5.0211970e-06 7.9466867e-07]]
Age : (8-12), conf = 0.968
1
Gender : Male, conf = 0.993
Age Output : [[6.9340258e-06 1.6550752e-04 9.0333790e-01 2.3891078e-02 7.2452247e-02
  1.3280979e-04 1.0463638e-05 3.1465931e-06]]
Age : (8-12), conf = 0.903
1
Gender : Male, conf = 0.996
Age Output : [[2.1980206e-06 5.5299432e-05 9.2458141e-01 1.6883872e-02 5.8390249e-02
  7.6557633e-05 8.3397263e-06 1.9388215e-06]]
Age : (8-12), conf = 0.925
1
Gender : Male, conf = 0.984
Age Output : [[8.3632676e-07 3.0895178e-05 9.8253340e-01 5.1183016e-03 1.2297507e-02
  1.6834349e-05 1.6102873e-06 4.6155839e-07]]
Age : (8-12), conf = 0.983
1
Gender : Male, conf = 0.995
Age Output : [[5.1183270e-07 2

# base.py

In [ ]:
import cv2 as cv2
from flask import Flask , Response , render_template

cam = cv2.VideoCapture(0)
app = Flask(__name__)

def stream():
    while 1 :
        __,frame = cam.read()
        imgencode = cv2.imencode('.jpg',frame)[1]
        stringData = imgencode.tostring()
        yield (b'--frame\r\n'b'Content-Type: text/plain\r\n\r\n'+stringData+b'\r\n')

@app.route('/video')
def video():
    return Response(stream(),mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/')
def main():
    return render_template('index.html')

if __name__ == "__main__":
    app.run()
    


# index.html

In [ ]:

<html>
    <img src="{{ url_for('video')}}">
</html>

# github에서 django 참조